# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score,make_scorer
from sklearn.neighbors import KNeighborsClassifier
import pickle
import warnings
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\libin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\libin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///C:\\Users\\libin\\OneDrive\\桌面\\processeddata.db')
df = pd.read_sql("SELECT * FROM df", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,19323,Days of heavy rains pounding the northern and ...,None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,19259,Musharraf said he feels satisfied that in a br...,None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,19321,"* Fodder relief - fodder (sometimes green, tho...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,19188,"In the scorched landscape of eastern Chad, man...",None,news,1,0,0,1,1,1,...,0,1,1,1,1,1,0,0,1,0
4,19320,"A small eruption took place in 1992, but cause...",None,news,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0


In [3]:
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    text=re.sub(r"[^a-zA-z]", " ",text.lower())
    tokens = word_tokenize(text)
    stemmed = [PorterStemmer().stem(word) for word in tokens if word not in stopwords.words("english")]
    
    return stemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ("vect", CountVectorizer(tokenizer = tokenize)),
    ("tfidf", TfidfTransformer()),
    ("clf", MultiOutputClassifier(RandomForestClassifier(verbose=False)))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.3, random_state= 42)
pipeline.fit(X_train, Y_train)


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001EE915EA160>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(verbose=False)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def get_score(y_true, y_pred, target_names):
    df = pd.DataFrame()
    for i,target in enumerate(target_names):
        accuracy = accuracy_score(y_true[:, i], y_pred[:, i])
        f1 = f1_score(y_true[:, i], y_pred[:, i])
        precision = precision_score(y_true[:, i], y_pred[:, i])
        recall = recall_score(y_true[:, i], y_pred[:, i])
        df = df.append({'index':target,'Accuracy':accuracy,'F1 Score':f1,'Precision':precision,'Recall':recall},ignore_index = True)
    return df

In [8]:
feature_name = Y.columns
y_train_pred = pipeline.predict(X_train)
y_train_true = np.array(Y_train)

In [9]:
result_train_df = get_score(y_train_true, y_train_pred, feature_name)
display(result_train_df)

,Accuracy,F1 Score,Precision,Recall,index
0,0.998024,0.998709,0.999211,0.998209,related
1,0.999232,0.997799,0.999370,0.996234,request
2,1.000000,1.000000,1.000000,1.000000,offer
3,0.999012,0.998823,0.999215,0.998432,aid_related
4,0.999780,0.998651,1.000000,0.997306,medical_help
5,0.999726,0.997257,1.000000,0.994530,medical_products
6,0.999890,0.998028,1.000000,0.996063,search_and_rescue
7,0.999726,0.992504,1.000000,0.985119,security
8,0.999945,0.999167,1.000000,0.998336,military
9,1.000000,1.000000,1.000000,1.000000,water


In [10]:
y_true = np.array(Y_test)
y_pred = pipeline.predict(X_test)
result_pred_df = get_score(y_true, y_pred, feature_name)
display(result_pred_df)

,Accuracy,F1 Score,Precision,Recall,index
0,0.820976,0.889800,0.838010,0.948412,related
1,0.900884,0.628242,0.823678,0.507764,request
2,0.995134,0.000000,0.000000,0.000000,offer
3,0.781406,0.724811,0.750584,0.700748,aid_related
4,0.925471,0.133929,0.616438,0.075125,medical_help
5,0.952491,0.177384,0.769231,0.100251,medical_products
6,0.973108,0.078947,0.750000,0.041667,search_and_rescue
7,0.982328,0.000000,0.000000,0.000000,security
8,0.967217,0.072464,0.588235,0.038610,military
9,0.957229,0.543716,0.865217,0.396414,water


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 20], 
              'clf__estimator__min_samples_split':[5, 10]}

cv = GridSearchCV(pipeline, param_grid = parameters)


In [12]:
tuned_model = cv.fit(X_train, Y_train)
print(tuned_model.best_params_)
print(cv.best_score_)

{'clf__estimator__min_samples_split': 5, 'clf__estimator__n_estimators': 20, 'tfidf__use_idf': False, 'vect__min_df': 5}
0.2606610635918757


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
y_tuned_pred = tuned_model.predict(X_test)
result_tuned_df = get_score(y_true, y_tuned_pred, feature_name)
display(result_tuned_df)


,Accuracy,F1 Score,Precision,Recall,index
0,0.821744,0.889207,0.844700,0.938666,related
1,0.896274,0.615019,0.792892,0.502329,request
2,0.995134,0.000000,0.000000,0.000000,offer
3,0.772570,0.718185,0.731416,0.705424,aid_related
4,0.925983,0.245431,0.562874,0.156928,medical_help
5,0.954155,0.266393,0.730337,0.162907,medical_products
6,0.973364,0.212121,0.583333,0.129630,search_and_rescue
7,0.981944,0.000000,0.000000,0.000000,security
8,0.969522,0.237179,0.698113,0.142857,military
9,0.958637,0.572185,0.853755,0.430279,water


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

parameters_kn = {'tfidf__use_idf':[True, False],
                 'clf__estimator__weights':['uniform','distance'],
                 'clf__estimator__leaf_size':[10,20]
                }
cv = GridSearchCV(pipeline, param_grid = parameters_kn)


kn_model = cv.fit(X_train, Y_train)
print(kn_model.best_params_)
print(cv.best_score_)

{'clf__estimator__leaf_size': 10, 'clf__estimator__weights': 'uniform', 'tfidf__use_idf': False}
0.246390834805514


In [16]:
y_kn_pred = kn_model.predict(X_test)
result_kn_df = get_score(y_true, y_kn_pred, feature_name)
display(result_kn_df)
display(result_kn_df.describe())

,Accuracy,F1 Score,Precision,Recall,index
0,0.792803,0.872014,0.823793,0.926231,related
1,0.877065,0.582971,0.661736,0.520963,request
2,0.995006,0.000000,0.000000,0.000000,offer
3,0.710975,0.599468,0.695921,0.526496,aid_related
4,0.926367,0.184397,0.613208,0.108514,medical_help
5,0.952235,0.227743,0.654762,0.137845,medical_products
6,0.972596,0.093220,0.550000,0.050926,search_and_rescue
7,0.982584,0.000000,0.000000,0.000000,security
8,0.967730,0.148649,0.594595,0.084942,military
9,0.953643,0.544081,0.739726,0.430279,water


,Accuracy,F1 Score,Precision,Recall
count,35.000000,35.000000,35.000000,35.000000
mean,0.938115,0.250176,0.519169,0.193310
std,0.064300,0.243787,0.267672,0.225030
min,0.710975,0.000000,0.000000,0.000000
25%,0.927776,0.052036,0.369444,0.027487
50%,0.954155,0.153061,0.594595,0.088235
75%,0.980983,0.437301,0.723695,0.307930
max,0.995390,0.872014,0.841365,0.926231


### 9. Export your model as a pickle file

In [17]:
pickle.dump(tuned_model, open('disaster_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.